Khai báo các thư viện cần thiết

In [3]:
import numpy as np
import cv2
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from skimage.transform import resize

Đọc dữ liệu

In [4]:
path = 'D:/Code/python/mat3533/practice07/data/face_data/'
ids = range(1, 16) # 15 people
states = ['centerlight', 'glasses', 'happy', 'leftlight',
'noglasses', 'normal', 'rightlight','sad',
'sleepy', 'surprised', 'wink' ]
prefix = 'subject'
surfix = '.png' #file extension is png
# open one picture to get the image's size
fn = prefix + '01.' + states[0] + surfix
im = cv2.imread(path + fn, 0)
h = im.shape[0] # hight
w = im.shape[1] # width
D = h * w
N = len(states)*15
print(N, D, h, w)
X = np.zeros((D, N))
# collect all data
count = 0
# there are 15 people
for person_id in range(1, 16):
    for state in states:
        # get name of each image file
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + surfix
        # open the file and read as grey image
        tmp = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)
        # then add image to dataset X
        X[:, count] = tmp.reshape(D)
        count += 1
y = np.repeat(range(1, 16), len(states))

165 77760 243 320


In [5]:
pca = PCA(n_components=100)
X_pca = pca.fit_transform(X.T)

Phân chia dữ  liệu theo tỉ lệ train:test=0.7:0.3

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

Chạy mô hình hồi quy Logistic nhiều chiều với dữ liệu đã giảm số chiều và in ra độ chính xác

In [7]:
from sklearn.linear_model import LogisticRegression

model_log = LogisticRegression()
model_log.fit(X_train, y_train)

y_pred_log = model_log.predict(X_test)
print(f"Prediction: \n{y_pred_log}")

accuracy_log = accuracy_score(y_test, y_pred_log)
print("Accuracy: ", accuracy_log)

Prediction: 
[13 11 12  6  9  3 15  5 10 14  2  8  2  7  3  3 13 10 14  2  7  2  2  1
  3 15  9  6 13 15 13  8  6  8  1 15  5  4  7 15  3 13 11  9 13  4  8  3
 12  7]
Accuracy:  0.98


Chạy mô hình Naive Bayes với dữ liệu đã giảm số chiều và in ra độ chính xác

In [8]:
from sklearn.naive_bayes import GaussianNB

model_nb = GaussianNB()
model_nb.fit(X_train, y_train)

y_pred_nb = model_nb.predict(X_test)
print(f"Prediction: \n{y_pred_nb}")

accuracy_nb = accuracy_score(y_test, y_pred_nb)
print("Accuracy: ", accuracy_nb)

Prediction: 
[ 8 11 12 12  9 15 15  5 10 14  2  8  2  8  3  3 15 11 14  2 15  2  2  1
  8 15  9  6 13 15 13 12  6  8  1  9  5  4 12 15  3 15 12  9 13  8 12  3
 12  8]
Accuracy:  0.7


Chạy mô hình ANN với dữ liệu đã giảm số chiều và in ra độ chính xác

In [9]:
from sklearn.neural_network import MLPClassifier

model_mlp = MLPClassifier()
model_mlp.fit(X_train, y_train)

y_pred_mlp = model_mlp.predict(X_test)
print(f"Prediction: \n{y_pred_mlp}")

accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
print("Accuracy: ", accuracy_mlp)

Prediction: 
[12  4 12 14 15 10 15  5 10 14  4 15  4  9  3  3  6 10 14  4 15  2 11 10
  9 10  9  6 13 15  6 15  6  9  1  7  5  4 15 15  3 13 12  9 13  8 15 10
 12  6]
Accuracy:  0.48


Đọc dữ liệu Test và đưa về cùng kích thước như trong ảnh dữ liệu (H = 320; W = 243)

In [10]:
path = 'D:/Code/python/mat3533/practice07/data/face_test_data/'
states = ['rightlight','sad', 'sleepy', 'surprised', 'wink']
prefix = 'picture'
surfix = '.png' #file extension is png
# open one picture to get the image's size
fn = prefix + surfix
im = cv2.imread(path + fn, 0)
N = 5
h = 320 # hight
w = 243 # width
D = h * w
print(N, D, h, w)
X_test_new = np.zeros((D, N))
for i in range(5):
    X_test_new[:, i] = resize(X_test_new[:, i].reshape((h, w)), (320, 243)).reshape(-1)
# collect all data
count = 0
for state in states:
    # get name of each image file
    fn = path + prefix + '_' + state + surfix
    # open the file and read as grey image
    tmp = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)
    # then add image to dataset X
    X_test_new[:, count] = tmp.reshape(D)
    count += 1

5 77760 320 243


In [11]:
from sklearn.linear_model import LogisticRegression

model_log = LogisticRegression()
model_log.fit(X.T, y)

y_pred_log = model_log.predict(X_test_new.T)
print(f"Prediction: \n{y_pred_log}")

Prediction: 
[15 15 15 15 15]


c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Dự đoán dữ liệu Test với mô hình hồi quy naive bayes

In [12]:
from sklearn.naive_bayes import GaussianNB

model_nb = GaussianNB()
model_nb.fit(X.T, y)

y_pred_nb = model_nb.predict(X_test_new.T)
print(f"Prediction: \n{y_pred_nb}")

Prediction: 
[15 15 15 15 15]


Dự đoán dữ liệu Test với mô hình ANN


In [13]:
from sklearn.neural_network import MLPClassifier

model_mlp = MLPClassifier()
model_mlp.fit(X.T, y)

y_pred_mlp = model_mlp.predict(X_test_new.T)
print(f"Prediction: \n{y_pred_mlp}")

Prediction: 
[10 10 10 10 10]
